1) comment on catégoriser le taux d'influence: t-stat
2) is movie significant ? -> p-value
3) same release year for several movies -> keep the one that is the most rated (number of ratings)
4) keep categories of 5 : top 5, most 5 popular movies which are not significant, bottom 5
5) remove all movies that have less than 100 ratings (to be determined maybe we discard too many movies or too litte...)
6) remove all characters that have an order below the median (or mean) of the number of character in the movie.
7) Careful : the number of movies with significant impact on baby names may drop a lot due to the point 5 and 6
8) the point 5 and 6 must be done also for the global study and not only for the website interactive heading (TO BE DISCUSSED!!)

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

# New code

In [2]:
folder_processed_data_path = './processed_data/'
processed_website_data_folder = './processed_data/website/'

In [3]:
# import the name by movie
name_by_movie_df = pd.read_csv(os.path.join(folder_processed_data_path, 'name_by_movie_ordered_pvalue_10_5_df.csv'))
name_by_movie_df.set_index(['wiki_ID', 'char_words', 'gender'], inplace=True)
display(name_by_movie_df.sample(2))

# import the movie dataframe
movie_df = pd.read_csv(os.path.join(folder_processed_data_path, 'movie_df.csv'))
movie_df.set_index(['wiki_ID'], inplace=True)
display(movie_df.sample(2))

,,,order,t_stat,p_value,slope_change
wiki_ID,char_words,gender,,,,
9002807,Marla,F,1.0,0.031859,0.975155,-0.000007
36956792,Beach,M,18.0,NaN,NaN,NaN


,mov_name,year,month,revenue,numVotes,averageRating,poster_url,IMDB_ID
wiki_ID,,,,,,,,
32805588,Amateur Crook,1937,NaN,NaN,64,5.0,NaN,tt0028574
9285798,Jaii,2004,3.0,NaN,48,5.4,NaN,tt2358060


## Compute `movie_impact` dataframe

In [4]:
# given name by user
chosen_name = 'trinity'

First, let's compute the 5 years with the most change in the babies for the chosen name.

In [5]:
# Remove useless columns
name_by_movie_web = name_by_movie_df.reset_index().copy(deep=True)
name_by_movie_web.drop(columns=['order', 't_stat'], inplace=True)
display(name_by_movie_web.sample(2))

,wiki_ID,char_words,gender,p_value,slope_change
67686,5215498,Stone,F,NaN,NaN
149412,26707166,Miria,F,NaN,NaN


In [6]:
# drop gender column
len_before = len(name_by_movie_web)

name_by_movie_web.reset_index(inplace=True, drop=True)
name_by_movie_web.drop_duplicates(subset=['wiki_ID', 'char_words'], keep='first', inplace=True)
name_by_movie_web.drop(columns=['gender'], inplace=True)
name_by_movie_web.set_index(['char_words', 'wiki_ID'], inplace=True)

len_after = len(name_by_movie_web)
print(f"length : {len_before} -> {len_after}")
print(f"Is the indexing of name_by_movie_web unique? {name_by_movie_web.index.is_unique}")
display(name_by_movie_web.sample(2))

length : 172906 -> 167984
Is the indexing of name_by_movie_web unique? True


,,p_value,slope_change
char_words,wiki_ID,,
Don,26154780,0.537117,-0.003706
Stevens,1190017,0.832911,0.000016


In [7]:
# drop NaN values
len_before = len(name_by_movie_web)
name_by_movie_web.dropna(subset=['p_value'], inplace=True)
len_after = len(name_by_movie_web)
print(f"length : {len_before} -> {len_after}")

length : 167984 -> 116464


In [8]:
# Add the year of release of the movie to the name_by_movie_web dataframe

needed_movie_info = movie_df.reset_index()[['wiki_ID', 'year', 'averageRating', 'numVotes']].copy(deep=True)

len_before_merge = len(name_by_movie_web)
name_by_movie_with_info = name_by_movie_web.reset_index().merge(needed_movie_info, on='wiki_ID', how='left').copy(deep=True) # merge the release year into the name_by_movie_web dataframe
len_after_merge = len(name_by_movie_with_info)
print(f"length : {len_before_merge} -> {len_after_merge}")

name_by_movie_with_info.set_index(['char_words', 'wiki_ID'], inplace=True)
display(name_by_movie_with_info.sample(2))

length : 116464 -> 116464


,,p_value,slope_change,year,averageRating,numVotes
char_words,wiki_ID,,,,,
Jeff,14737421,0.792988,0.000113,1991,7.0,201
Mitch,2377503,0.990742,-0.000002,2005,6.7,103826


In [9]:
# set the char_names in lowercase
name_by_movie_with_info.reset_index(inplace=True)
name_by_movie_with_info['char_words'] = name_by_movie_with_info['char_words'].str.lower()
name_by_movie_with_info.set_index(['char_words', 'wiki_ID'], inplace=True)
print(f"Length of name_by_movie_with_info : {len(name_by_movie_with_info)}")
display(name_by_movie_with_info.sample(2))

Length of name_by_movie_with_info : 116464


,,p_value,slope_change,year,averageRating,numVotes
char_words,wiki_ID,,,,,
gray,566713,0.389508,-0.000155,1987,6.7,103839
lloyd,2984133,0.883618,0.000063,2006,3.2,1448


In [10]:
# create the movie_impact dataframe
columns = ['name', 'status', 'movie_id']
movie_impact_df = pd.DataFrame(columns=columns)
display(movie_impact_df)

,name,status,movie_id


### Functions to compute TOP/BOTTOM/INSIGN

In [11]:
def compute_top_movies(name, chosen_name_movies_top_df, movie_impact_df):
    for i in range(5): # from top1 to top5
        if(not chosen_name_movies_top_df.empty):
            # get the movies release the year with the highest positive variation
            top_i_year = chosen_name_movies_top_df.iloc[-1].year.astype(int)
            top_i_year_chosen_name_movies = chosen_name_movies_top_df.query(f'year == {top_i_year}').copy(deep=True)

            # keep only the three most popular movies
            top_i_year_chosen_name_movies.sort_values(by=['numVotes'], ascending=False, inplace=True)
            top_i_3_chosen_name_movies = top_i_year_chosen_name_movies.iloc[:3].copy(deep=True)

            # add the the three popular movies to the movie_impact_df
            for index, row in top_i_3_chosen_name_movies.iterrows():
                movie_impact_df.loc[len(movie_impact_df)] = {'name': name, 'status': 't', 'movie_id': index}

            # remove the found movies from the chosen_name_movies_top_df to avoid picking them again for next iterations
            chosen_name_movies_top_df.drop(top_i_year_chosen_name_movies.index, inplace=True)

            # remove also the movies release in the frame [-5 years, +5 years] of the top1_year
            chosen_name_movies_top_df.query(f'year < {top_i_year - 5} or year > {top_i_year + 5}', inplace=True)

In [12]:
def compute_bottom_movies(name, chosen_name_movies_bottom_df, movie_impact_df):
    for i in range(5): # from bottom1 to bottom5
        if(not chosen_name_movies_bottom_df.empty):
            # get the movies release the year with the highest negative variation
            bottom_i_year = chosen_name_movies_bottom_df.iloc[-1].year.astype(int)
            bottom_i_year_chosen_name_movies = chosen_name_movies_bottom_df.query(f'year == {bottom_i_year}').copy(deep=True)

            # keep only the three most popular movies
            bottom_i_year_chosen_name_movies.sort_values(by=['numVotes'], ascending=False, inplace=True)
            bottom_i_3_chosen_name_movies = bottom_i_year_chosen_name_movies.iloc[:3].copy(deep=True)

            # add the the three popular movies to the movie_impact_df
            for index, row in bottom_i_3_chosen_name_movies.iterrows():
                movie_impact_df.loc[len(movie_impact_df)] = {'name': name, 'status': 'b', 'movie_id': index}

            # remove the found movies from the chosen_name_movies_bottom_df to avoid picking them again for next iterations
            chosen_name_movies_bottom_df.drop(bottom_i_year_chosen_name_movies.index, inplace=True)

            # remove also the movies release in the frame [-5 years, +5 years] of the bottom_i_year
            chosen_name_movies_bottom_df.query(f'year < {bottom_i_year - 5} or year > {bottom_i_year + 5}', inplace=True)

In [13]:
def compute_insign_movies(name, chosen_name_movies_insign_df, movie_impact_df):
    for i in range(5): # from insign1 to insign5
        if(not chosen_name_movies_insign_df.empty):
            # get the movies release the year with the highest rating and number of votes
            insign_i_year = chosen_name_movies_insign_df.iloc[-1].year.astype(int)
            insign_i_year_chosen_name_movies = chosen_name_movies_insign_df.query(f'year == {insign_i_year}').copy(deep=True)

            # keep only the three most popular movies
            insign_i_3_chosen_name_movies = insign_i_year_chosen_name_movies.iloc[:3].copy(deep=True)

            # add the the three popular movies to the movie_impact_df
            for index, row in insign_i_3_chosen_name_movies.iterrows():
                movie_impact_df.loc[len(movie_impact_df)] = {'name': name, 'status': 'i', 'movie_id': index}

            # remove the found movies from the chosen_name_movies_top_df to avoid picking them again for next iterations
            chosen_name_movies_insign_df.drop(insign_i_year_chosen_name_movies.index, inplace=True)

            # remove also the movies release in the frame [-5 years, +5 years] of the top1_year
            chosen_name_movies_insign_df.query(f'year < {insign_i_year - 5} or year > {insign_i_year + 5}', inplace=True)

### Special case for a single name

In [ ]:
# get the movie containing the chosen name
chosen_name_movies_df = name_by_movie_with_info.loc[chosen_name, :].copy(deep=True)
display(chosen_name_movies_df)

In [ ]:
chosen_name_movies_df.sort_values(by=['slope_change'], inplace=True)
chosen_name_movies_top_df = chosen_name_movies_df.query('(slope_change > 0) and (p_value < 0.1)').copy(deep=True)
display(chosen_name_movies_top_df)

Do the loop to do the computation for top1 to top5

In [ ]:
compute_top_movies(chosen_name, chosen_name_movies_top_df, movie_impact_df)

display(movie_impact_df)

In [ ]:
chosen_name_movies_bottom_df = chosen_name_movies_df.query('(slope_change <= 0) and (p_value < 0.1)').copy(deep=True)
chosen_name_movies_bottom_df.sort_values(by=['slope_change'], ascending=False, inplace=True)
display(chosen_name_movies_bottom_df)

In [ ]:
compute_bottom_movies(chosen_name, chosen_name_movies_bottom_df, movie_impact_df)

display(movie_impact_df)

In [ ]:
remaining_chosen_name_movies_df = pd.concat([chosen_name_movies_top_df, chosen_name_movies_bottom_df])
chosen_name_movies_insign_df = remaining_chosen_name_movies_df.query('p_value > 0.1').copy(deep=True)
chosen_name_movies_insign_df.sort_values(by=['averageRating', 'numVotes'], inplace=True)
display(chosen_name_movies_insign_df)

In [ ]:
compute_insign_movies(chosen_name, chosen_name_movies_insign_df, movie_impact_df)
display(movie_impact_df)

In [ ]:
# check if there movie duplicates in the movie_impact_df
print(f"number of duplicates in movie_impact_df : {movie_impact_df.duplicated(subset=['movie_id']).sum()}")

### Generalize for all the names

In [14]:
# get all the names in the name_by_movie_with_info dataframe
names = name_by_movie_with_info.index.get_level_values(0).unique().tolist()
print(names)
print(f"Number of names : {len(names)}")

['linda', 'henry', 'duke', 'williams', 'sheila', 'arthur', 'eldon', 'lewis', 'leon', 'hannibal', 'rick', 'rachael', 'tyrell', 'roy', 'bryant', 'sebastian', 'lamarr', 'van', 'bart', 'lyle', 'buddy', 'johnson', 'von', 'sam', 'howard', 'harriett', 'lili', 'jim', 'don', 'dorothy', 'hunter', 'frank', 'jeffrey', 'raymond', 'ben', 'paul', 'barbara', 'sandy', 'mike', 'lyndon', 'barry', 'kimberly', 'murray', 'nicole', 'andy', 'benny', 'gary', 'cassandra', 'joyce', 'jennifer', 'merrick', 'edward', 'campbell', 'william', 'king', 'princess', 'robert', 'stephen', 'wallace', 'stewart', 'bruce', 'malcolm', 'prince', 'isabelle', 'james', 'alexander', 'gordon', 'vicki', 'carl', 'bob', 'harvey', 'alfred', 'cooper', 'morgan', 'miles', 'robin', 'harriet', 'lady', 'max', 'charles', 'haven', 'ivy', 'wilson', 'grayson', 'nora', 'julie', 'madison', 'burton', 'chase', 'eric', 'roosevelt', 'jerry', 'thompson', 'leland', 'mary', 'anderson', 'herbert', 'young', 'susan', 'foster', 'walter', 'bertha', 'carter', 'th

In [15]:
iter = 0
for name in names:
    print(f"Number of name treated: {iter}", end='\r', flush=True)
    # get the movie containing the chosen name
    chosen_name_movies_df = name_by_movie_with_info.loc[name, :].copy(deep=True)

    # sort the movies by slope_change, get movies with significant p_value and positive slope_change and compute the top 5
    chosen_name_movies_df.sort_values(by=['slope_change'], inplace=True)
    chosen_name_movies_top_df = chosen_name_movies_df.query('(slope_change > 0) and (p_value < 0.1)').copy(deep=True)
    # display(chosen_name_movies_top_df)
    compute_top_movies(name, chosen_name_movies_top_df, movie_impact_df)

    # sort the movies by slope_change, get movies with significant p_value and negative slope_change and compute the bottom 5
    chosen_name_movies_bottom_df = chosen_name_movies_df.query('(slope_change <= 0) and (p_value < 0.1)').copy(deep=True)
    chosen_name_movies_bottom_df.sort_values(by=['slope_change'], ascending=False, inplace=True)
    compute_bottom_movies(name, chosen_name_movies_bottom_df, movie_impact_df)

    # get the remaining movies and filter to keep only the insignificant ones and compute the insign 5
    # remaining_chosen_name_movies_df = pd.concat([chosen_name_movies_top_df, chosen_name_movies_bottom_df])
    chosen_name_movies_insign_df = chosen_name_movies_df.query('p_value > 0.1').copy(deep=True)
    chosen_name_movies_insign_df.sort_values(by=['averageRating', 'numVotes'], inplace=True)
    compute_insign_movies(name, chosen_name_movies_insign_df, movie_impact_df)
    
    iter = iter + 1

In [16]:
# movie_impact_df.set_index(['name', 'status'], inplace=True)
display(movie_impact_df)

# Export DataFrame to a CSV file in the processed data folder
# movie_impact_df.reset_index(drop=True).sort_values(by=['name', 'status']).to_csv(os.path.join(processed_website_data_folder, 'movie_impact.csv'), index=False)

,name,status,movie_id
0,linda,t,4070671
1,linda,t,9956825
2,linda,t,2212664
3,linda,t,28288095
4,linda,t,10364456
...,...,...,...
27834,serene,i,36544941
27835,kathrine,i,36546325
27836,zelma,i,36598217
27837,beauregard,t,36699915


### Import `movie_impact` dataframe to avoid timeconsuming computation

In [17]:
# import the movie_impact_df to avoid the time-consuming computation of movie_impact_df
imported_movie_impact_df = pd.read_csv(os.path.join(processed_website_data_folder, 'movie_impacts.csv'))
imported_movie_impact_df.set_index(['name', 'status'], inplace=True)
display(imported_movie_impact_df)

FileNotFoundError: [Errno 2] No such file or directory: './processed_data/website/movie_impact.csv'

In [ ]:
# Check for NaN values in the name column
contains_nan = imported_movie_impact_df.reset_index()['name'].isna().any()

if contains_nan:
    print("The `name` column contains NaN values.")
else:
    print("The `name` column does not contain NaN values.")

In [18]:
# drop NaN values in the name column
print(f"Length of imported_movie_impact_df before dropping NaN values : {len(imported_movie_impact_df)}")
imported_movie_impact_df.reset_index(inplace=True)
imported_movie_impact_df.dropna(subset=['name'], inplace=True)
imported_movie_impact_df.set_index(['name', 'status'], inplace=True)
print(f"Length of imported_movie_impact_df after dropping NaN values : {len(imported_movie_impact_df)}")

display(imported_movie_impact_df)
movie_impact_df = imported_movie_impact_df.copy(deep=True)

NameError: name 'imported_movie_impact_df' is not defined

## Compute `name_per_year` dataframe

In [19]:
# import the babynames dataframe
name_per_year = pd.read_csv(os.path.join(folder_processed_data_path, 'baby_name_df.csv'))
name_per_year.drop(columns='number', inplace=True)
name_per_year.set_index(['name', 'year'], inplace=True)
print(f"Is the indexing of name_per_year unique? {name_per_year.index.is_unique}")
display(name_per_year.sample(2))
print(f"Length of the name_per_year_df : {len(name_per_year)}")

Is the indexing of name_per_year unique? True


,,percentage
name,year,
Payshence,2009,0.000136
Shaylin,2004,0.001918


Length of the name_per_year_df : 1903290


In [20]:
# round the percentage values to reduce the size of the future csv file for the web
name_per_year['percentage'] = name_per_year['percentage'].round(6)

# set the names in lowercases
name_per_year.reset_index(inplace=True)
name_per_year['name'] = name_per_year['name'].str.lower()
name_per_year.set_index(['name', 'year'], inplace=True)

display(name_per_year.sample(2))
print(f"Length of the name_per_year_df : {len(name_per_year)}")

,,percentage
name,year,
cortavious,1991,0.000026
yennifer,2013,0.000057


Length of the name_per_year_df : 1903290


In [21]:
# filter the name_per_year_df to keep only the names in the movie_impact_df
name_per_year_filtered = name_per_year.loc[movie_impact_df['name'].unique().tolist(), :].copy(deep=True)
name_per_year = name_per_year_filtered.copy(deep=True)

In [22]:
# Let's sort the dataframe name_per_year to anticipate the ploting
name_per_year.sort_values(by=['name', 'year'], inplace=True)
display(name_per_year.head(15))

percentage
name  year            
aadam 1987    0.000028
      1988    0.000028
      1993    0.000082
      1994    0.000055
      1995    0.000056
      1996    0.000028
      1997    0.000028
      1998    0.000112
      1999    0.000028
      2000    0.000055
      2002    0.000193
      2003    0.000163
      2004    0.000135
      2005    0.000054
      2006    0.000131

Let's check if the names present in name_per_year dataframe are also present in the name_by_movie dataframe

In [23]:
# Identify names in name_per_year not present in name_by_movie
values_only_in_name_per_year = name_per_year.reset_index()[~name_per_year.reset_index()['name'].isin(name_by_movie_with_info.reset_index()['char_words'])]['name'].unique()

print(f"Number of unique names in name_per_year  = {len(name_per_year.reset_index()['name'].unique())}")

# Display the result
print("Values in name_per_year but not present in name_by_movie:")
print(values_only_in_name_per_year)
print(f"Number of names missing  = {len(values_only_in_name_per_year)}")

Number of unique names in name_per_year  = 6034
Values in name_per_year but not present in name_by_movie:
[]
Number of names missing  = 0


In [24]:
# Identify names in name_by_movie not present in name_per_year
values_only_in_name_by_movie = name_by_movie_with_info.reset_index()[~name_by_movie_with_info.reset_index()['char_words'].isin(name_per_year.reset_index()['name'])]['char_words'].unique()

print(f"Number of unique names in name_by_movie_with_info  = {len(name_by_movie_with_info.reset_index()['char_words'].unique())}")

# Display the result
print("Values in name_by_movie_with_info but not present in name_per_year:")
print(values_only_in_name_by_movie)
print(f"Number of names missing  = {len(values_only_in_name_by_movie)}")

Number of unique names in name_by_movie_with_info  = 6034
Values in name_by_movie_with_info but not present in name_per_year:
[]
Number of names missing  = 0


Compute the number of names in the three dataframes in order to compare

In [25]:
# compute how many unique names are there in the dataframes to compare
names_in_name_per_year = name_per_year.index.get_level_values(0).unique().tolist()
print(f"Number of unique names in name_per_year = {len(names_in_name_per_year)}")
names_in_movie_impact = movie_impact_df.reset_index()['name'].unique().tolist()
print(f"Number of unique names in movie_impact_df = {len(names_in_movie_impact)}")
names_in_name_by_movie_with_info = name_by_movie_with_info.reset_index()['char_words'].unique().tolist()
print(f"Number of unique names in name_by_movie_with_info = {len(names_in_name_by_movie_with_info)}")

Number of unique names in name_per_year = 6034
Number of unique names in movie_impact_df = 6034
Number of unique names in name_by_movie_with_info = 6034


## Compute `movies` dataframe
This dataset contains the information relative to the movie given its `wiki_ID`.

In [26]:
print(f"Number of movies in movie_impact_df : {len(movie_impact_df['movie_id'].unique())}")
# keep only the movies in the movie_df that are in the movie_impact_df
simplified_movie_df = movie_df.loc[movie_impact_df['movie_id'].unique().tolist(), :].copy(deep=True)
print(f"Number of movies kept in movie_df: {len(simplified_movie_df)}")

# remove useless columns
simplified_movie_df.drop(columns=['month', 'revenue'], inplace=True)
simplified_movie_df.rename(columns={'averageRating': 'rating', 'numVotes': 'votes', 'IMDB_ID': 'imdb_id'}, inplace=True)

display(simplified_movie_df.sample(2))

Number of movies in movie_impact_df : 13944
Number of movies kept in movie_df: 13944


,mov_name,year,votes,rating,poster_url,imdb_id
wiki_ID,,,,,,
13632871,La Femme Musketeer,2004,1130,5.5,NaN,tt0341855
420532,Lucía y el sexo,2001,38213,7.0,/gvErAg9bv7xOO3B40gottv6Bqip.jpg,tt0254455


## Export the three datasets

In [27]:
# Export movie_impact.csv
display(movie_impact_df.reset_index().sample(2))
movie_impact_df.reset_index().to_csv(os.path.join(processed_website_data_folder, 'movie_impacts.csv'), index=False)

# Export name_per_year.csv
display(name_per_year.reset_index().sample(2))
name_per_year.reset_index().to_csv(os.path.join(processed_website_data_folder, 'name_per_year.csv'), index=False)

# Export movie.csv
display(simplified_movie_df.reset_index().sample(2))
simplified_movie_df.reset_index().to_csv(os.path.join(processed_website_data_folder, 'movies.csv'), index=False)

,index,name,status,movie_id
27653,27653,ottilie,i,31546554
17915,17915,vera,t,513090


,name,year,percentage
3483,adam,2014,0.149120
162043,emilie,1906,0.011319


,wiki_ID,mov_name,year,votes,rating,poster_url,imdb_id
3919,972816,Clockwatchers,1997,6141,6.6,/gFMklZMFVso3qnbnNFMYWPKesC0.jpg,tt0118866
6040,99454,Patton,1970,106438,7.9,/jrOqV1oStB0Eaa5EmACmKbzIewy.jpg,tt0066206
